### Update config of Fires data to Layer Manger 3

We want to fecth the Active Fires from Resource Watch but the layerCongig is outdated to use with layer Manger 3. We need to adapt the layer config

#### Get the Fires dataset

In [16]:
import LMIPy as lmi
import json

In [3]:
l = lmi.Layer('ec7ce55d-5c77-486b-b86b-e2e46adde9f6')

In [5]:
l.attributes['layerConfig']

{'account': 'rw-nrt',
 'body': {'maxzoom': 18,
  'minzoom': 3,
  'layers': [{'type': 'cartodb',
    'options': {'sql': "SELECT cartodb_id, the_geom, acq_date, acq_time, acq_date::text as date_str, acq_time::text as time_str, frp, latitude, longitude, the_geom_webmercator, CASE WHEN acq_date>= (CURRENT_DATE - interval '7 day') AND acq_date< (CURRENT_DATE - interval '6 day') THEN 7 WHEN acq_date>= (CURRENT_DATE - interval '6 day') AND acq_date< (CURRENT_DATE - interval '5 day') THEN 6 WHEN acq_date>= (CURRENT_DATE - interval '5 day') AND acq_date< (CURRENT_DATE - interval '4 day') THEN 5 WHEN acq_date>= (CURRENT_DATE - interval '4 day') AND acq_date< (CURRENT_DATE - interval '3 day') THEN 4 WHEN acq_date>= (CURRENT_DATE - interval '3 day') AND acq_date< (CURRENT_DATE - interval '2 day') THEN 3 WHEN acq_date>= (CURRENT_DATE - interval '2 day') AND acq_date< (CURRENT_DATE - interval '1 day') THEN 2 WHEN acq_date>= (CURRENT_DATE - interval '1 day') THEN 1 ELSE -1 END AS days_ago FROM vnp14i

In [6]:
### Fecth another CARTO dataset that has the new config
l3 = lmi.Layer('f72e7a11-e0e2-4cbc-a645-5593d3f050fa')
l3.attributes['layerConfig']

{'source': {'type': 'vector',
  'provider': {'type': 'carto',
   'account': 'wri-01',
   'layers': [{'options': {'sql': "SELECT cartodb_id, the_geom_webmercator, name, to_number(size_ha,'999999999999') as area_ha, category, issuedate as date_create FROM khm_protected_areas",
      'type': 'cartodb'}}]}},
 'render': {'layers': [{'type': 'fill',
    'source-layer': 'layer0',
    'paint': {'fill-opacity': 0.6, 'fill-color': '#3182BD'}},
   {'type': 'line',
    'source-layer': 'layer0',
    'paint': {'line-opacity': 1, 'line-color': '#FFF', 'line-width': 0.25}}]},
 'type': 'vector',
 'version': '3.0'}

In [7]:
atts = l.attributes
lc = atts['layerConfig']
new_lc = {
    "type": "vector",
    "version": "3.0", ## Layer Manager Version tracking, optional
    "source": { ## 'where do I fetch the data from'
        "type": "vector",
        "provider": { ##carto stuff
            "type": "carto",
            'account': lc['account'],
            'layers': lc['body']['layers']
        }
    },
    "render": { ## 'how should it look'
        "layers": lc['body']['vectorLayers'] #...vectorLayers key from carto config 
    }
}

In [15]:
new_config = {
    **atts,
    "provider": 'vector',
    'layerConfig': new_lc
}
new_config

{'name': 'June 24, 2021 - June 30, 2021 All Fires',
 'slug': 'All-Fires-Past-Week_10',
 'dataset': '64c948a6-5e34-4ef2-bb69-6a6535967bd5',
 'description': 'This layer shows fires detected in the past week.',
 'application': ['rw'],
 'iso': [],
 'provider': 'vector',
 'type': 'layer',
 'userId': '5ba001878e311b7e3718740f',
 'default': False,
 'protected': False,
 'published': True,
 'env': 'production',
 'layerConfig': {'type': 'vector',
  'version': '3.0',
  'source': {'type': 'vector',
   'provider': {'type': 'carto',
    'account': 'rw-nrt',
    'layers': [{'type': 'cartodb',
      'options': {'sql': "SELECT cartodb_id, the_geom, acq_date, acq_time, acq_date::text as date_str, acq_time::text as time_str, frp, latitude, longitude, the_geom_webmercator, CASE WHEN acq_date>= (CURRENT_DATE - interval '7 day') AND acq_date< (CURRENT_DATE - interval '6 day') THEN 7 WHEN acq_date>= (CURRENT_DATE - interval '6 day') AND acq_date< (CURRENT_DATE - interval '5 day') THEN 6 WHEN acq_date>= (CURR

In [18]:
with open('all_fires_lmv3.json', 'w') as fp:
    json.dump(new_config, fp)